In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
import hook_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [2]:
model: HookedTransformer = HookedTransformer.from_pretrained("gpt2-large")

data = load_dataset("stas/openwebtext-10k", split="train")
# data = [item for i, item in enumerate(data["text"]) if len(item) > 2000 and i < 5000]
data = data['text']

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


Repo card metadata block was not found. Setting CardData to empty.


### Data

In [3]:
with open('data/gpt2_large_spectrum_bins.pkl', 'rb') as f:
    bin_edges = pickle.load(f)
bin_edges

lower_bounds = torch.tensor(bin_edges[:-1]).contiguous().cuda()

In [8]:
layer, neuron = 31, 892

hook_name = f'blocks.{layer}.mlp.hook_post'
hooks = [(hook_name, hook_utils.save_activation)]
max_range = np.array([i for i in range(model.cfg.n_ctx)])
an_tokens = torch.tensor([model.to_single_token(' an'), model.to_single_token(' AN'), model.to_single_token(' An')]).cuda()

prompt_indices = []
token_indices = []
tokens = []
next_tokens = []
next_is_ans = []
acts = []
bins = []
bin_counts = torch.zeros(lower_bounds.shape[0] + 1).cuda()

for i, prompt in tqdm(enumerate(data)):
    if len(prompt) < 20:
        continue
    all_tokens = model.to_tokens(prompt, truncate=True)[0]
    if all_tokens.shape[0] < 2:
        continue
    
    # Don't include the final token because we need the next token
    current_tokens = all_tokens[:-1]

    with model.hooks(hooks):
        model(current_tokens)

    current_acts = model.hook_dict[hook_name].ctx['activation'][0, :, neuron].contiguous()

    prompt_indices.append(np.array([i]*current_tokens.shape[0]))
    token_indices.append(max_range[:current_tokens.shape[0]])
    tokens.append(current_tokens.cpu().numpy())
    next_tokens.append(all_tokens[1:].cpu().numpy())
    acts.append(current_acts.cpu().numpy())
    next_is_ans.append((torch.isin(all_tokens[1:], an_tokens)).cpu().numpy())

    bin_indices = torch.searchsorted(lower_bounds, current_acts)
    bins.append(bin_indices.cpu().numpy())
    bin_counts += torch.bincount(bin_indices, minlength=bin_counts.shape[0])
    if i % 100 == 0:
        if (bin_counts > 20).all():
            break

0it [00:00, ?it/s]

In [9]:
prompt_indices = np.concatenate(prompt_indices)
token_indices = np.concatenate(token_indices)
tokens = np.concatenate(tokens)
next_tokens = np.concatenate(next_tokens)
next_is_ans = np.concatenate(next_is_ans)
acts = np.concatenate(acts)

In [10]:
df = pd.DataFrame({
    'prompt_index': prompt_indices,
    'token_index': token_indices,
    'token': tokens,
    'next_token': next_tokens,
    'next_is_an': next_is_ans,
    'activation': acts,
})

In [33]:
# Sort the DataFrame
df.sort_values('activation', inplace=True)
trimmed_df = df.iloc[0:-349]
df_shuffled = trimmed_df.sample(frac=1, random_state=0).reset_index(drop=True)

bin_labels = np.linspace(df_shuffled['activation'].min(), df_shuffled['activation'].max(), 21)
df_shuffled['L31N892 Activation'] = pd.cut(df_shuffled['activation'], bins=bin_labels)

# df_shuffled['L31N892 Activation'], bin_edges = pd.cut(df_shuffled['activation'], bins=20, labels=False, retbins=True)

sample_df = df_shuffled.groupby('L31N892 Activation').apply(lambda x: x.sample(n=min(len(x), 50), random_state=0)).reset_index(drop=True)


# Count items in each bin
bin_counts = sample_df['L31N892 Activation'].value_counts().sort_index()

print(bin_counts)

/tmp/ipykernel_7394/901762627.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



L31N892 Activation
(-0.17, 0.18]     50
(0.18, 0.529]     50
(0.529, 0.879]    50
(0.879, 1.229]    50
(1.229, 1.578]    50
(1.578, 1.928]    50
(1.928, 2.277]    50
(2.277, 2.627]    50
(2.627, 2.977]    50
(2.977, 3.326]    50
(3.326, 3.676]    50
(3.676, 4.026]    50
(4.026, 4.375]    50
(4.375, 4.725]    50
(4.725, 5.075]    50
(5.075, 5.424]    50
(5.424, 5.774]    50
(5.774, 6.123]    50
(6.123, 6.473]    50
(6.473, 6.823]    50
Name: count, dtype: int64


In [34]:
bin_labels

array([-0.17004077,  0.17959953,  0.52923984,  0.87888015,  1.22852045,
        1.57816076,  1.92780107,  2.27744137,  2.62708168,  2.97672198,
        3.32636229,  3.6760026 ,  4.0256429 ,  4.37528321,  4.72492351,
        5.07456382,  5.42420413,  5.77384443,  6.12348474,  6.47312504,
        6.82276535])

In [35]:
with open('data/gpt2_large_spectrum_v2.pkl', 'wb') as f:
    pickle.dump(sample_df, f)

with open('data/gpt2_large_spectrum_bins_v2.pkl', 'wb') as f:
    pickle.dump(bin_labels, f)

In [ ]:
with open('data/gpt2_large_spectrum_v2.pkl', 'rb') as f:
    sample_df = pickle.load(f)

with open('data/gpt2_large_spectrum_bins_v2.pkl', 'rb') as f:
    bin_edges = pickle.load(f)

In [ ]:
grouped = sample_df.groupby('L31N892 Activation', as_index=False)['next_is_an'].agg(['sum', 'count'])
grouped['Next token is \' an\''] = grouped['sum'] / grouped['count']
grouped['Next token is not \' an\''] = 1 - grouped['Next token is \' an\'']

grouped_melted = grouped.melt(id_vars=['L31N892 Activation'], value_vars=['Next token is \' an\'', 'Next token is not \' an\''], 
                            var_name='status', value_name='Proportion of Labels')
fig = px.area(grouped_melted, x='L31N892 Activation', y='Proportion of Labels', color='status', line_shape='spline')
tickvals = np.arange(len(bin_edges) - 1)
ticktext = [f"{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}" for i in tickvals]

fig.update_layout(
    yaxis=dict(range=[0, 1]),
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ),
)
fig.show()

In [ ]:
merged

,L31N892 Activation,next_token,count,total_count,Proportion of Labels,Next Token,Scaled Proportion of Labels
93,1,281,3,100,0.03,an,0.000181
166,2,281,6,100,0.06,an,0.000108
231,3,281,15,100,0.15,an,0.000093
289,4,281,17,100,0.17,an,0.000043
349,5,281,23,100,0.23,an,0.000026
...,...,...,...,...,...,...,...
41,0,2354,1,100,0.01,outside,0.009910
42,0,2407,1,100,0.01,quite,0.009910
326,4,2456,1,100,0.01,words,0.000003
134,1,2479,1,100,0.01,age,0.000060


In [ ]:

# Group by 'bucket' and 'next_token', then calculate proportions
# Separates into groups by bucket and next_token, adds size columns, then converts back to dataframe via reset index
grouped_by_bucket_and_next_token = sample_df.groupby(['L31N892 Activation', 'next_token']).size().reset_index(name='count')
total = grouped_by_bucket_and_next_token.groupby('L31N892 Activation')['count'].sum().reset_index(name='total_count')
merged = pd.merge(grouped_by_bucket_and_next_token, total, on='L31N892 Activation')
merged['activation'] = sample_df['activation']
merged['Proportion of Labels'] = merged['count'] / merged['total_count']


# Sort data by commonality of next token
total_counts = merged.groupby('next_token')['Proportion of Labels'].sum().reset_index()
sorted_tokens = total_counts.sort_values('Proportion of Labels', ascending=False)['next_token'].tolist()

merged['next_token'] = pd.Categorical(merged['next_token'], categories=sorted_tokens, ordered=True)
merged.sort_values(['next_token', 'L31N892 Activation'], inplace=True)
merged['Next Token'] = merged['next_token'].apply(lambda x: model.to_single_str_token(x))

# Create plot
fig = px.area(merged, x='L31N892 Activation', y='Proportion of Labels', color='Next Token', line_shape='spline', title='Proportions of next token by L31N892')
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ),
    yaxis=dict(range=[0, 1]))
fig.show()

# Merge in other forms of an?
# Second chart: scale each bucket by proportion of dataset that falls into that bucket, e.g. proportion * 0.01. Should result in a bell curve
# Third chart: Activation value * Pr(neuron being in bucket)

In [ ]:
def get_scaled_proportion(row):
    bucket_size = bucket_sizes[bucket_sizes['L31N892 Activation'] == row['L31N892 Activation']].iloc[0, -1].item()
    scaling_value = bucket_size / total
    return row['Proportion of Labels'] * scaling_value

bucket_sizes = df_shuffled.groupby('L31N892 Activation').size().reset_index()
total = bucket_sizes.iloc[:, -1].sum()

merged['Scaled Proportion of Labels'] = merged.apply(get_scaled_proportion, axis=1)


In [ ]:
fig = px.area(merged, x='L31N892 Activation', y='Scaled Proportion of Labels', color='Next Token', line_shape='spline', title='Proportions of next token by L31N892')

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ))
fig.show()

In [ ]:
merged

,L31N892 Activation,next_token,count,total_count,activation,Proportion of Labels,Next Token,Expected Value
93,1,281,3,100,-0.127699,0.03,an,-0.003831
166,2,281,6,100,1.252725,0.06,an,0.075163
231,3,281,15,100,1.340770,0.15,an,0.201115
289,4,281,17,100,1.326652,0.17,an,0.225531
349,5,281,23,100,2.233818,0.23,an,0.513778
...,...,...,...,...,...,...,...,...
41,0,2354,1,100,-0.141081,0.01,outside,-0.001411
42,0,2407,1,100,-0.009845,0.01,quite,-0.000098
326,4,2456,1,100,2.262700,0.01,words,0.022627
134,1,2479,1,100,0.810942,0.01,age,0.008109


In [ ]:
# Third chart: Activation value * Pr(neuron being in bucket)

def get_scaled_proportion(row):
    bucket_size = bucket_sizes[bucket_sizes['L31N892 Activation'] == row['L31N892 Activation']].iloc[0, -1].item()
    scaling_value = bucket_size / total
    return row['Proportion of Labels'] * scaling_value

bucket_sizes = df_shuffled.groupby('L31N892 Activation').size().reset_index()
total = bucket_sizes.iloc[:, -1].sum()

merged['Expected Value'] = merged['activation'] * merged['Scaled Proportion of Labels']


fig = px.area(merged, x='L31N892 Activation', y='Expected Value', color='Next Token', line_shape='spline')

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ))
fig.show()